In [1]:
# Heroes Of Pymoli Data Analysis

In [2]:
import pandas as pd
import numpy as np
import json
import os

In [3]:
with open('purchase_data.json') as f:
    purchase_data = json.load(f)

In [68]:
purchase_data_df = pd.DataFrame.from_dict(purchase_data, orient='columns')

In [5]:
# Player Count

In [6]:
players = purchase_data_df.groupby('SN').nunique()
total_players = players['SN'].count()
tp = {'Total Players': [str(total_players)]}
total_players_df = pd.DataFrame(data=tp)
total_players_df

,Total Players
0,573


In [7]:
# Purchasing Analysis

In [8]:
unique_items = purchase_data_df.groupby('Item Name').nunique()
total_unique = unique_items['Item Name'].count()

In [9]:
average_price = np.round(purchase_data_df['Price'].mean(), decimals=2)

In [10]:
total_purchases = purchase_data_df['Item Name'].count()

In [11]:
total_revenue = purchase_data_df['Price'].sum()

In [12]:
p_a = {'Number of Unique Items': [str(total_unique)],
       'Average Price': [str(average_price)],
       'Number of Purchases': [str(total_purchases)],
       'Total Revenue': [str(total_revenue)]}
purchasing_analysis_df = pd.DataFrame(data=p_a)
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,2.93,780,2286.33


In [13]:
# Gender Demographics

In [14]:
gender = purchase_data_df.drop_duplicates('SN')
gender_count = gender['Gender'].value_counts()

In [15]:
gender_count = gender_count.to_frame().reset_index()

In [16]:
gender_count = gender_count.rename(columns={'index': 'Genders', 'Gender': 'Total Count'})

In [17]:
gender_count = gender_count.set_index('Genders')

In [18]:
gender_final = (gender_count['Total Count']/573*100).round(2)
gender_count['Percentage of Players'] = gender_final
gender_count

,Total Count,Percentage of Players
Genders,,
Male,465,81.15
Female,100,17.45
Other / Non-Disclosed,8,1.40


In [19]:
# Purchasing Analysis (Gender)

In [20]:
g_a = purchase_data_df.groupby('Gender')
gender_analysis = g_a['Gender'].count()
gender_analysis_df = gender_analysis.to_frame(name=None)
ga = gender_analysis_df.rename(columns={'Gender': 'Purchase Count'})

In [21]:
gender_average_price = np.round(purchase_data_df.pivot_table(purchase_data_df, 'Gender'), decimals = 2)
gap = gender_average_price.rename(columns={'Price': 'Average Purchase Price'})
gap = gap.drop(columns=['Age', 'Item ID'])

In [22]:
gender_total_price = np.round(purchase_data_df.pivot_table(purchase_data_df, 'Gender', aggfunc=sum), decimals = 2)
gtp = gender_total_price.rename(columns={'Price': 'Total Purchase Price'})
gtp = gtp.drop(columns=['Age', 'Item ID'])

In [23]:
final_gender = pd.concat([ga, gap, gtp], axis=1)

In [24]:
final_gender['Normalized Totals']=np.round((final_gender['Total Purchase Price']/
                                            gender_count['Total Count']),decimals=2)
final_gender

,Purchase Count,Average Purchase Price,Total Purchase Price,Normalized Totals
Gender,,,,
Female,136,2.82,382.91,3.83
Male,633,2.95,1867.68,4.02
Other / Non-Disclosed,11,3.25,35.74,4.47


In [25]:
# Age Demographics

In [26]:
unique_demo = purchase_data_df.pivot_table(purchase_data_df, 'SN')

In [66]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 500]
age_group = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
unique_demo['Age Bracket'] = pd.cut(unique_demo['Age'], bins, labels = age_group)
age_bracket = unique_demo['Age Bracket'].value_counts()
age_bracket = age_bracket.to_frame(name=None)
age_bracket['Percentage of Players']=np.round(age_bracket['Age Bracket']/573*100, decimals=2)
age_bracket = age_bracket.rename(columns={'Age Bracket': 'Total Count'})
age_bracket

,Total Count,Percentage of Players
20-24,259,45.20
15-19,100,17.45
25-29,87,15.18
30-34,47,8.20
35-39,27,4.71
10-14,23,4.01
<10,19,3.32
40+,11,1.92


In [28]:
# Purchasing Analysis (Age)

In [58]:
bins1 = [0, 9, 14, 19, 24, 29, 34, 39, 500]
age_group1 = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
purchase_data_df['Age Bracket'] = pd.cut(purchase_data_df['Age'], bins1, labels = age_group1)
purchasing_item = purchase_data_df

In [65]:
p_item1 = purchasing_item.groupby('Age Bracket').count()
p_item1 = p_item1.drop(columns=['Age', 'Gender','Item ID','Price','SN'])
p_item1 = p_item1.rename(columns={'Item Name': 'Purchase Count'})

In [64]:
p_item2 = np.round(purchasing_item.groupby('Age Bracket').mean(),decimals=2)
p_item2 = p_item2.drop(columns=['Age','Item ID'])
p_item2 = p_item2.rename(columns={'Price': 'Average Purchase Price'})

In [63]:
p_item3 = purchasing_item.groupby('Age Bracket').sum()
p_item3 = p_item3.drop(columns=['Age','Item ID'])
p_item3 = p_item3.rename(columns={'Price': 'Total Purchase Value'})

In [67]:
p_final = pd.concat([p_item1,p_item2,p_item3], axis=1,sort=True)
p_final['Normalized Total']=np.round(p_final['Total Purchase Value']/age_bracket['Total Count'], decimals=2)
p_final

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Total
Age Bracket,,,,
<10,28,2.98,83.46,4.39
10-14,35,2.77,96.95,4.22
15-19,133,2.91,386.42,3.86
20-24,336,2.91,978.77,3.78
25-29,125,2.96,370.33,4.26
30-34,64,3.08,197.25,4.20
35-39,42,2.84,119.40,4.42
40+,17,3.16,53.75,4.89


In [30]:
# Top Spenders

In [31]:
spender_sort = np.round(purchase_data_df.pivot_table(purchase_data_df, 'SN', aggfunc=sum), decimals=2)
spender_sort = spender_sort.sort_values(by=['Price'], ascending=False)
spender_sort = spender_sort.drop(columns=['Age', 'Item ID'])
spender_sort = spender_sort.rename(columns={'Price': 'Total Purchase Value'})

In [32]:
spender = purchase_data_df.groupby('SN')
spender = spender['Price'].mean()
spender = np.round(spender.to_frame(name=None),decimals=2)

In [33]:
spender2 = purchase_data_df.groupby('SN')
spender2 = spender2['SN'].count()
spender2 = np.round(spender2.to_frame(name=None),decimals=2)

In [34]:
sp = pd.concat([spender_sort, spender, spender2], axis=1, sort=True)
final_sp = sp.sort_values(by=['Total Purchase Value'], ascending=False)
final_sp = final_sp.rename(columns={'Price': 'Average Purchase Price', 'SN':'Purchase Count'})
final_spenders = final_sp.head()
final_spenders

,Total Purchase Value,Average Purchase Price,Purchase Count
Undirrala66,17.06,3.41,5
Saedue76,13.56,3.39,4
Mindimnya67,12.74,3.18,4
Haellysu29,12.73,4.24,3
Eoda93,11.58,3.86,3


In [35]:
# Most Popular Items

In [36]:
popular_items = purchase_data_df.groupby(['Item ID','Item Name']).count().reset_index()
popular = popular_items.sort_values('Price', ascending=False)
popular = popular.set_index(['Item ID','Item Name'])
popular = popular.drop(columns=['Age', 'Gender', 'Price'])
popular = popular.rename(columns={'SN': 'Purchase Count'})

In [37]:
popular_items1 = purchase_data_df.groupby(['Item ID', 'Item Name'])
popular_items1 = popular_items1['Price'].mean()
popular_items1 = np.round(popular_items1.to_frame(name=None),decimals=2)
popular_items1 = popular_items1.rename(columns={'Price':'Item Price'})

In [38]:
popular_items2 = purchase_data_df.groupby(['Item ID', 'Item Name'])
popular_items2 = popular_items2['Price'].sum()
popular_items2 = np.round(popular_items2.to_frame(name=None),decimals=2)
popular_items2 = popular_items2.rename(columns={'Price':'Total Purchase Value'})

In [39]:
popular_final = pd.concat([popular,popular_items1,popular_items2], axis=1,sort=True)
popular_final = popular_final.sort_values(by=['Purchase Count'], ascending=False)
popular_final = popular_final.head()
popular_final

,,Purchase Count,Age Bracket,Item Price,Total Purchase Value
Item ID,Item Name,,,,
39,"Betrayal, Whisper of Grieving Widows",11,11,2.35,25.85
84,Arcane Gem,11,11,2.23,24.53
31,Trickster,9,9,2.07,18.63
175,Woeful Adamantite Claymore,9,9,1.24,11.16
13,Serenity,9,9,1.49,13.41


In [40]:
# Most Profitable Items

In [41]:
profitable_items = purchase_data_df.groupby(['Item ID','Item Name']).sum().reset_index()
profit = profitable_items.sort_values('Price', ascending=False)
profit = profit.set_index(['Item ID','Item Name']).head()
profit = profit.drop(columns=['Age'])
profit = profit.rename(columns={'Price': 'Total Purchase Value'})

In [42]:
profit1 = purchase_data_df.groupby(['Item ID', 'Item Name'])
profit1 = profit1['Price'].mean()
profit1 = np.round(profit1.to_frame(name=None),decimals=2)
profit1 = profit1.rename(columns={'Price':'Item Price'})

In [43]:
profit2 = purchase_data_df.groupby(['Item ID', 'Item Name'])
profit2 = profit2['SN'].count()
profit2 = np.round(profit2.to_frame(name=None),decimals=2)
profit2 = profit2.rename(columns={'SN':'Purchase Count'})

In [44]:
profit_final = pd.concat([profit,profit1,profit2], axis=1,sort=True)
profit_final = profit_final.sort_values(by=['Total Purchase Value'], ascending=False)
profit_final = profit_final.head()
profit_final

,,Total Purchase Value,Item Price,Purchase Count
Item ID,Item Name,,,
34,Retribution Axe,37.26,4.14,9
115,Spectral Diamond Doomblade,29.75,4.25,7
32,Orenmir,29.70,4.95,6
103,Singed Scalpel,29.22,4.87,6
107,"Splitter, Foe Of Subtlety",28.88,3.61,8
